# 04.4- Agents



Let's take a look at agents, and expand the capabilities of LLMs.

### AgentType

There are several types of agents, each optimized for what we need. An agent can store past chat information or can enforce each call to the LLM as an independent one.  
It is up to you to decide what you need in your future projects, but here we will focus on agents capable of maintaining a chat.

### Tools

The `tools` are a set of options one uses to give further capabilities to the agent: parsing wikipedia, using google for searches, etc..  
Several models are trained with a 1-2 year delay in data, like chatgpt-3.5. It was only trained with data up to 2022.

In [4]:
#!pip install numexpr
#!pip install langchainhub
#!pip install langchain_openai

  Using cached langchain_openai-0.0.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached tiktoken-0.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached regex-2023.12.25-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached langchain_openai-0.0.5-py3-none-any.whl (29 kB)
Using cached tiktoken-0.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached regex-2023.12.25-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (785 kB)


In [2]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.agents import load_tools

ModuleNotFoundError: No module named 'langchain_openai'

In [2]:
prompt = hub.pull("hwchase17/structured-chat-agent")

In [7]:
model = ChatOpenAI()

tools = load_tools(["wikipedia","llm-math"], llm=model) 

agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [8]:
agent_executor.invoke({"input": "hi"})

{'input': 'hi', 'output': 'Hello! How can I assist you today?'}

In [9]:
agent_executor.invoke({"input": "Can you recommend some tasty Portuguese dishes?"}, handle_parsing_errors=True)
#                       "system": "You are a helpful AI Masterchef."})

TypeError: WikipediaQueryRun._run() got an unexpected keyword argument 'title'

In [ ]:
from langchain.agents import load_tools
from langchain.agents import create_structured_chat_agent
from langchain.agents import AgentType
from langchain_openai import OpenAI
from langchain_core.prompts.chat import ChatPromptTemplate

In [ ]:
llm = OpenAI(temperature=0.1)

In [ ]:
tools = load_tools(["wikipedia","llm-math"], llm=llm) 

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot specialized on food."),
    ("input", "Can you recommend me some good Portuguese dishes?"),
])

In [ ]:
agent = create_structured_chat_agent(llm,
                                     tools,
                                     template
                                    )

In [ ]:
agent.invoke({"input":"Who sang at the Barcelona Olympics?"})

In [ ]:
llm=OpenAI(temperature=0.1)

# We want our agent to handle math
Gv
# We want the agent to keep a memory of what is "talked"
memory = ConversationBufferMemory(memory_key="chat_history")

#agent = create_react_agent(tools, 
#                           llm, "What are some tasty Portuguese cuisine recipes?")

In [ ]:
agent.invoke(input="What are some tasty Portuguese cuisine recipes?", handle_parsing_errors=True)

In [ ]:
response = agent.invoke(input="Can you describe the recipe for 'Sopa da Pedra'?", handle_parsing_errors=True)

In [ ]:
Markdown(response['output'])

---
### Other applications: Analyzing documents

In [ ]:
import requests

# Let's get our Shakespeare back
response = requests.get("https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

# There is a size limit for inputs
shakespeare = response.text[0:10000]

In [ ]:
from langchain.chains import AnalyzeDocumentChain

In [ ]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm)

In [ ]:
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

qa_document_chain.run(
    input_document=shakespeare,
    question="What is this work?",
)

In [ ]:
qa_document_chain.run(
    input_document=shakespeare,
    question="Describe the main protagonist of this work.",
)

---
### PDF documents

In [ ]:
!pip install pypdf

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
!wget https://media.wizards.com/images/magic/resources/rules/EN_MTGM14_PrintedRulebook_LR.pdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("EN_MTGM14_PrintedRulebook_LR.pdf")
pages = loader.load_and_split()

In [ ]:
qa_document_chain.run(
    input_document=pages,
    question="What are the phases of the turn?",
)

In [ ]:
pages